In [35]:
import torch
import torch.nn as nn
import torch.optim as optim

import cv2 as cv
import mediapipe as mp

import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from IPython.display import display, Image
from sklearn.preprocessing import MinMaxScaler, Normalizer

from tqdm import tqdm
import os
import sys

import json
from glob import glob
from collections import OrderedDict

# Define the Neural Network

In [36]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        # First fully connected layer
        self.fc1 = nn.Linear(input_size, hidden_size)  
        # Activation function
        self.relu = nn.ReLU()
        # Output layer
        self.fc2 = nn.Linear(hidden_size, num_classes)  

    def forward(self, x):
        # Forward pass: compute predicted y
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define model parameters
input_size = 63
hidden_size = 128  # You can adjust this value
num_classes = 2

# Instantiate the model
model = NeuralNet(input_size, hidden_size, num_classes)

# Define Loss Function and Optimizer

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # lr is the learning rate

# Training the Model

In [38]:
def train(model, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, target)
            
            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

# You would call this function with your data loader
# train(model, your_data_loader, epochs=10)

# Evaluation

In [39]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Call this with your test data loader
# test(model, your_test_data_loader)

# test

In [40]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pandas import DataFrame

In [41]:
df = pd.read_json("processed_data.json")
df['label'] = df['label'].replace({'A': 0, 'B': 1})
df.head()

/tmp/ipykernel_11113/2288009784.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'A': 0, 'B': 1})


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z12,z13,z14,z15,z16,z17,z18,z19,z20,label
0,0.226127,0.234860,0.228681,0.226548,0.238650,0.216837,0.215509,0.216044,0.217245,0.210701,...,0.226009,0.243140,0.204783,0.217565,0.232866,0.235311,0.222603,0.220795,0.220288,0
1,0.198414,0.229695,0.249374,0.252871,0.265572,0.236031,0.234447,0.229043,0.227424,0.219662,...,0.229932,0.254994,0.200085,0.227337,0.256310,0.250004,0.222089,0.236451,0.249909,0
2,0.207967,0.222814,0.230614,0.241090,0.248850,0.241206,0.229363,0.218375,0.213111,0.235535,...,0.227831,0.247175,0.220036,0.229214,0.240060,0.245323,0.240766,0.235146,0.231505,0
3,0.215926,0.236616,0.238391,0.242547,0.266568,0.229986,0.225167,0.219033,0.218083,0.219858,...,0.223613,0.245314,0.206596,0.221143,0.233069,0.238586,0.222188,0.225017,0.224854,0
4,0.194080,0.225672,0.244753,0.251601,0.267459,0.232277,0.235588,0.226857,0.221222,0.217419,...,0.232845,0.249419,0.200438,0.226675,0.254053,0.240751,0.217912,0.230999,0.243238,0


In [42]:
import torch
from torch.utils.data import Dataset, DataLoader

class PandasDataset(Dataset):
    def __init__(self, dataframe, feature_cols, label_col, transform=None):
        self.dataframe = dataframe
        self.feature_cols = feature_cols
        self.label_col = label_col
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        features = row[self.feature_cols].values.astype(float)
        label = row[self.label_col]
        
        if self.transform:
            features = self.transform(features)
        
        return torch.tensor(features, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Example usage
feature_cols = [f"x{i}" for i in range(21)] + [f"y{i}" for i in range(21)] + [f"z{i}" for i in range(21)]
label_col = 'label'
dataset = PandasDataset(df, feature_cols, label_col)


In [43]:
# dataloader = DataLoader(dataset, batch_size=100, shuffle=True, num_workers=4)

from torch.utils.data import random_split

# Define split ratios
train_ratio = 0.8
test_ratio = 1 - train_ratio

# Compute lengths
dataset_size = len(dataset)
train_size = int(train_ratio * dataset_size)
test_size = dataset_size - train_size

# Split dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])



In [44]:
# Define DataLoader parameters
batch_size = 8
num_workers = 0  # Adjust based on your system

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [45]:
train(model, train_loader, 40)

Train Epoch: 0 [0/852 (0%)]	Loss: 0.687674
Train Epoch: 0 [800/852 (93%)]	Loss: 0.453562
Train Epoch: 1 [0/852 (0%)]	Loss: 0.353410
Train Epoch: 1 [800/852 (93%)]	Loss: 0.117124
Train Epoch: 2 [0/852 (0%)]	Loss: 0.043202
Train Epoch: 2 [800/852 (93%)]	Loss: 0.042902
Train Epoch: 3 [0/852 (0%)]	Loss: 0.060777
Train Epoch: 3 [800/852 (93%)]	Loss: 0.017108
Train Epoch: 4 [0/852 (0%)]	Loss: 0.015590
Train Epoch: 4 [800/852 (93%)]	Loss: 0.006397
Train Epoch: 5 [0/852 (0%)]	Loss: 0.026039
Train Epoch: 5 [800/852 (93%)]	Loss: 0.003972
Train Epoch: 6 [0/852 (0%)]	Loss: 0.021379
Train Epoch: 6 [800/852 (93%)]	Loss: 0.017435
Train Epoch: 7 [0/852 (0%)]	Loss: 0.005215
Train Epoch: 7 [800/852 (93%)]	Loss: 0.006607
Train Epoch: 8 [0/852 (0%)]	Loss: 0.008899
Train Epoch: 8 [800/852 (93%)]	Loss: 0.002138
Train Epoch: 9 [0/852 (0%)]	Loss: 0.014611
Train Epoch: 9 [800/852 (93%)]	Loss: 0.004703
Train Epoch: 10 [0/852 (0%)]	Loss: 0.002753
Train Epoch: 10 [800/852 (93%)]	Loss: 0.001391
Train Epoch: 11 [0/

In [46]:
test(model, test_loader)


Test set: Average loss: 0.0000, Accuracy: 214/214 (100%)



In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiClassModel(nn.Module):
    def __init__(self):
        super(MultiClassModel, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(63, 128)  # input layer (63) -> hidden layer (128)
        self.fc2 = nn.Linear(128, 24)   # hidden layer (128) -> output layer (24)

    def forward(self, x):
        # Define forward pass
        x = F.relu(self.fc1(x))      # activation function for hidden layer
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # log softmax for output

# Instantiate the model
model = MultiClassModel()

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train(model, device, loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader), loss.item()))

# Then you would call train in a loop for each epoch
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'
model.to('cpu')

for epoch in range(1, 10):  # loop over the dataset multiple times
    train(model, device, train_loader, optimizer, epoch)

Train Epoch: 1 [0/852 (0%)]	Loss: 3.274391
Train Epoch: 1 [800/852 (93%)]	Loss: 0.634232
Train Epoch: 2 [0/852 (0%)]	Loss: 0.592219
Train Epoch: 2 [800/852 (93%)]	Loss: 0.434054
Train Epoch: 3 [0/852 (0%)]	Loss: 0.320384
Train Epoch: 3 [800/852 (93%)]	Loss: 0.130343
Train Epoch: 4 [0/852 (0%)]	Loss: 0.191026
Train Epoch: 4 [800/852 (93%)]	Loss: 0.114458
Train Epoch: 5 [0/852 (0%)]	Loss: 0.119213
Train Epoch: 5 [800/852 (93%)]	Loss: 0.050240
Train Epoch: 6 [0/852 (0%)]	Loss: 0.057566
Train Epoch: 6 [800/852 (93%)]	Loss: 0.027130
Train Epoch: 7 [0/852 (0%)]	Loss: 0.038113
Train Epoch: 7 [800/852 (93%)]	Loss: 0.037230
Train Epoch: 8 [0/852 (0%)]	Loss: 0.077610
Train Epoch: 8 [800/852 (93%)]	Loss: 0.024302
Train Epoch: 9 [0/852 (0%)]	Loss: 0.006672
Train Epoch: 9 [800/852 (93%)]	Loss: 0.013514
